In [1]:
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
from scipy import stats
from sklearn.decomposition import PCA
from scipy import spatial # para calcular similaridade


In [2]:
import random
import sys
import copy
from igraph import *
import time
import numpy as np
import pickle

In [3]:
g = Graph.Read_Ncol("CA-GrQc.txt", directed = False)
A = g.get_edgelist()
nxg = nx.Graph(A)

In [4]:
with open('centralidades.txt', 'rb') as f:
    my_list2 = pickle.load(f)

In [15]:
X = np.array(my_list2)
X = X.transpose()
X = stats.zscore(X)
X.shape

(5242, 7)

In [16]:
varience_values = []

pca = PCA(n_components=1,svd_solver='full')
teste = pca.fit_transform(X)
explained_varience = pca.explained_variance_ratio_.sum()
print(float(pca.explained_variance_ratio_.sum()*100))
print(pca.explained_variance_ratio_,len(pca.explained_variance_ratio_))
print(pca.singular_values_,len(pca.singular_values_))  

52.5657647754
(array([ 0.52565765]), 1)
(array([ 138.88297853]), 1)


In [17]:
lista_teste = teste.tolist()
len(lista_teste)

5242

In [18]:
#  a partir daqui é so pegar os index igual no que ja tem
flat_list = []
for sublist in lista_teste:
    for item in sublist:
        flat_list.append(item)
        
len(flat_list)
# print(flat_list[:50])

5242

In [19]:
n = 50
vindex = [i for i in range(g.vcount())]
aflat, seeds_PCA = (list(x) for x in zip(*sorted(zip(flat_list, vindex), reverse=True)))
print(seeds_PCA[:50])
# len(vindex)

[1037, 577, 11, 288, 53, 207, 20, 108, 186, 101, 279, 72, 295, 103, 147, 244, 545, 365, 363, 1088, 346, 77, 100, 369, 465, 353, 396, 265, 1032, 303, 300, 1066, 282, 457, 296, 1515, 230, 36, 1091, 1733, 159, 102, 269, 2065, 1284, 359, 285, 315, 543, 375]


In [20]:
my_list = X
# my_list[0] = grau
# my_list[1] = betweenness
# my_list[2] = pagerank
# my_list[3] = closeness
# ...etc
aflat, seeds_grau = (list(x) for x in zip(*sorted(zip(my_list[:,0], vindex), reverse=True)))
aflat, seeds_betweennes = (list(x) for x in zip(*sorted(zip(my_list[:,1], vindex), reverse=True)))
aflat, seeds_pagerank = (list(x) for x in zip(*sorted(zip(my_list[:,2], vindex), reverse=True)))
aflat, seeds_closeness = (list(x) for x in zip(*sorted(zip(my_list[:,3], vindex), reverse=True)))
aflat, seeds_eigenvector_centrality = (list(x) for x in zip(*sorted(zip(my_list[:,4], vindex), reverse=True)))
aflat, seeds_harmonic_centrality = (list(x) for x in zip(*sorted(zip(my_list[:,5], vindex), reverse=True)))
aflat, seeds_load_centrality = (list(x) for x in zip(*sorted(zip(my_list[:,6], vindex), reverse=True)))
# aflat, seeds_katz_centrality = (list(x) for x in zip(*sorted(zip(my_list[7], vindex), reverse=True)))
l = [seeds_grau,seeds_betweennes,seeds_pagerank,seeds_closeness,seeds_eigenvector_centrality,seeds_harmonic_centrality,seeds_load_centrality]
print(l[0][:50])

[101, 295, 279, 103, 77, 72, 296, 288, 265, 100, 159, 285, 1284, 282, 302, 275, 263, 262, 269, 300, 303, 301, 277, 267, 259, 221, 297, 280, 261, 299, 294, 290, 264, 284, 273, 1037, 577, 293, 292, 291, 289, 286, 283, 278, 276, 274, 270, 266, 545, 1278]


In [21]:
for i in l:
    print(1 - spatial.distance.cosine(seeds_PCA[:50], i[:50]))

0.620295963129
0.424147168956
0.568983424659
0.442329556589
0.658290517801
0.556445822039
0.425286123296


In [22]:
# X = np.array(my_list)
# X = X.transpose()
print('linhas x colunas',X.shape)

kendall = []
kendall2 = []

for i in range((X.shape[1])):
    for j in range((X.shape[1])):
        if (i != j):
            tau, p_value = kendalltau(X[:,i],X[:,j])
            if (p_value > 0):
                if tau>0.1:
                    kendall.append((tau,i,j))
                else:
                    kendall2.append((tau,i,j))
            if (p_value < 0):
                print ("menor",i,j,tau,(p_value)*1000)

print(kendall)
print(' ')
print(kendall2)

('linhas x colunas', (5242, 7))
[(0.33544209326008195, 1, 4), (0.33544209326008195, 4, 1), (0.33510950875196593, 4, 6), (0.33510950875196593, 6, 4)]
 
[(0.079003534095916539, 2, 3), (0.032928892628109163, 2, 4), (0.08723391799168731, 2, 5), (0.079003534095916539, 3, 2), (0.032928892628109163, 4, 2), (0.08723391799168731, 5, 2)]


In [23]:
def calcula_PCA(col1,col2): # pca entre duas colunas
    
    Y = np.array([X[:,col1],X[:,col2]])
    Y = Y.transpose()
#     print (Y.shape)
#     print (Y,type(Y),type(Y[0]),len(Y[0]))
    
    pca = PCA(n_components=1,svd_solver='full')
    teste = pca.fit_transform(Y)
    explained_varience = pca.explained_variance_ratio_.sum()
    print(float(pca.explained_variance_ratio_.sum()*100))
    lista_teste = teste.tolist()
    flat_list = []
    for sublist in lista_teste:
        for item in sublist:
            flat_list.append(item)
    aflat, seeds_PCA = (list(x) for x in zip(*sorted(zip(flat_list, vindex), reverse=True)))
    return seeds_PCA

In [24]:
# calcula_PCA(1,4)[:50]
for i in kendall2:
    print (i)
    print(calcula_PCA(i[-2],i[-1])[:50])
    print('--'*10)

(0.079003534095916539, 2, 3)
57.2350583211
[108, 1037, 577, 11, 295, 186, 53, 103, 101, 279, 1733, 1032, 20, 1243, 31, 365, 288, 363, 72, 207, 450, 1088, 457, 346, 369, 315, 370, 100, 396, 296, 1091, 77, 244, 465, 545, 45, 265, 230, 27, 2009, 375, 1066, 543, 984, 746, 159, 398, 2065, 285, 322]
--------------------
(0.032928892628109163, 2, 4)
65.977822331
[295, 101, 279, 103, 288, 72, 296, 100, 77, 265, 159, 285, 269, 282, 263, 275, 302, 262, 303, 300, 301, 261, 277, 259, 267, 280, 290, 297, 294, 273, 264, 299, 284, 293, 292, 291, 289, 286, 283, 278, 276, 274, 266, 270, 271, 260, 108, 1037, 577, 545]
--------------------
(0.08723391799168731, 2, 5)
58.4498622044
[108, 1037, 577, 11, 295, 186, 103, 53, 101, 279, 1733, 1032, 20, 288, 72, 1243, 31, 365, 363, 207, 450, 1088, 457, 346, 100, 369, 315, 370, 296, 77, 396, 244, 1091, 545, 465, 265, 45, 230, 27, 1066, 2009, 375, 159, 543, 984, 746, 398, 285, 2065, 322]
--------------------
(0.079003534095916539, 3, 2)
57.2350583211
[108, 1037, 5